In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

import random, os, pathlib, time
from tqdm import tqdm
from sklearn import datasets

In [2]:
device = torch.device("cuda:1")
# device = torch.device("cpu")

## MNIST dataset

In [3]:
import mylibrary.datasets as datasets
import mylibrary.nnlib as tnn

In [4]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

# train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [5]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [6]:
input_size = 784
output_size = 10

In [7]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [8]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [9]:
batch_size = 50
train_loader = data.DataLoader(dataset=train_dataset,
                                    num_workers=4, 
                                    batch_size=batch_size, 
                                    shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                    num_workers=4, 
                                    batch_size=batch_size, 
                                    shuffle=False)

In [10]:
class DistanceTransform(nn.Module):
    
    def __init__(self, input_dim, num_centers):
        super().__init__()
        self.input_dim = input_dim
        self.num_centers = num_centers
        
        self.centers = torch.randn(num_centers, input_dim)/3.
        self.centers = nn.Parameter(self.centers)
        
    def forward(self, x):
        x = x[:, :self.input_dim]
        dists = torch.cdist(x, self.centers)
#         print(dists.shape)
        dists = dists/np.sqrt(self.input_dim) ### correction to make diagonal of unit square 1 in nD space
        return dists
    
    def reverse(self, x):
        ### here x is the distance, so weighted distance is the reversed mode
        ##### for a single input, y = mean(x_i * center_i : for i in range(num_centers))
        pass

In [11]:
dt = DistanceTransform(784, 784)

In [12]:
list(dt.parameters())

[Parameter containing:
 tensor([[-0.5206, -0.2707,  0.4184,  ...,  0.2339, -0.1133,  0.3311],
         [-0.8345,  0.4408,  0.3426,  ..., -0.1288,  0.0946,  0.3073],
         [ 0.4055,  0.3256,  0.3914,  ...,  0.2980, -0.2194, -0.3904],
         ...,
         [-0.1354, -0.2526, -0.2932,  ..., -0.1841,  0.1694, -0.2163],
         [ 0.2216,  0.2443,  0.0117,  ..., -0.1474,  0.2455, -0.6724],
         [ 0.1658,  0.2606, -0.0729,  ..., -0.0178, -0.0808, -0.0677]],
        requires_grad=True)]

In [13]:
dt(torch.randn(2, 784)).shape

torch.Size([2, 784])

In [14]:
x=train_dataset[[3,7]][0]
dists = dt(x)
x.std(), dists.std()

(tensor(0.3944), tensor(0.1574, grad_fn=<StdBackward>))

In [15]:
dists.max(), dists.min()

(tensor(0.8295, grad_fn=<MaxBackward1>),
 tensor(0.4508, grad_fn=<MinBackward1>))

In [16]:
dt.centers.shape

torch.Size([784, 784])

In [17]:
### single variable inverse
x = torch.randn(1, 2)
x

tensor([[ 0.2484, -1.4337]])

In [18]:
centers = torch.randn(2, 2) ## output_dim / num_centers, input_dim
y = torch.cdist(x, centers.t())  
y.shape

torch.Size([1, 2])

In [19]:
y

tensor([[2.2636, 1.5797]])

In [20]:
weighted_sum = 0
weight = (1/y).sum()
for i in range(len(centers)):
    weighted_sum += centers[i]*(1/y[0,i])/weight
weighted_sum

tensor([ 0.4111, -1.2158])

In [21]:
##### test for inverse
x = torch.randn(2, 4)

In [22]:
centers = torch.randn(4, 4) ## output_dim / num_centers, input_dim
y = torch.cdist(x, centers)  
y.shape

torch.Size([2, 4])

In [23]:
y

tensor([[3.2547, 2.3560, 3.5674, 2.1832],
        [1.8720, 2.1646, 1.7646, 1.8540]])

In [24]:
(y.unsqueeze(2)*centers).shape

torch.Size([2, 4, 4])

In [25]:
z = (y.unsqueeze(2)*centers.t()).sum(dim=2)/y.sum(dim=1, keepdim=True)
z.shape

torch.Size([2, 4])

In [26]:
z

tensor([[-0.8764,  0.2324, -0.2695,  0.4836],
        [-0.7481,  0.3169, -0.1978,  0.6095]])

In [27]:
x

tensor([[ 1.9518,  0.0419,  0.0577,  0.8557],
        [-0.5095,  0.8375, -0.1642, -0.9393]])

In [40]:
model = nn.Sequential(
#                 DistanceTransform(784, 784),
                nn.BatchNorm1d(784),
                nn.Linear(784, 200),
                nn.LeakyReLU(),
                nn.Linear(200, 50),
                nn.LeakyReLU(),
                nn.Linear(50, 10))
model.to(device)

Sequential(
  (0): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=784, out_features=200, bias=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=200, out_features=50, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Linear(in_features=50, out_features=10, bias=True)
)

In [41]:
optimizer = optim.Adam(list(model.parameters()), 
                            lr=0.0003, weight_decay=1e-15)
criterion = nn.CrossEntropyLoss()

In [42]:
index = 0
train_accs, test_accs = [], []
for epoch in tqdm(list(range(40))):
    train_acc = 0
    train_count = 0
    for xx, yy in train_loader:
        xx, yy = xx.to(device), yy.to(device)
        yout = model(xx)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in test_loader:
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> MAX Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 0:0,  Loss:0.29208049178123474


  2%|▎         | 1/40 [00:03<02:16,  3.49s/it]

Train Acc:81.78%, Test Acc:85.08%

Epoch: 1:0,  Loss:0.3728877902030945


  5%|▌         | 2/40 [00:06<02:12,  3.50s/it]

...

Train Acc:97.09%, Test Acc:88.74%

Epoch: 38:0,  Loss:0.09802291542291641


 98%|█████████▊| 39/40 [02:10<00:03,  3.37s/it]

Train Acc:97.07%, Test Acc:88.63%

Epoch: 39:0,  Loss:0.10097994655370712


100%|██████████| 40/40 [02:13<00:00,  3.33s/it]

Train Acc:97.25%, Test Acc:88.52%

	-> MAX Train Acc 97.255 ; Test Acc 89.01


In [35]:
# -> MAX Train Acc 87.678 ; Test Acc 85.87  ### after distance transformation
# -> MAX Train Acc 97.255 ; Test Acc 89.01  ### normal nn

In [36]:
model.eval()
dists = model[0](train_dataset[[0,3,7]][0].to(device))
dists = model[1](dists)
model.train()
dists.shape

torch.Size([3, 784])

In [37]:
dists.mean()

tensor(0.8673, device='cuda:1', grad_fn=<MeanBackward0>)

In [38]:
dists.std()

tensor(1.3996, device='cuda:1', grad_fn=<StdBackward>)

In [39]:
dists[0]

tensor([0.6848, 1.3316, 0.7227, 0.5346, 1.1709, 1.3659, 0.5240, 0.4794, 0.7535,
        1.0121, 1.1492, 0.5022, 0.9565, 1.4633, 0.6267, 1.2688, 0.4046, 1.2330,
        1.0712, 0.6266, 0.6519, 0.6877, 1.1010, 0.4282, 0.6463, 1.4526, 0.8620,
        0.5370, 0.5170, 0.8810, 0.4652, 0.6724, 0.5919, 0.9145, 0.7398, 1.3985,
        0.8407, 0.4311, 0.6553, 1.4876, 1.3120, 1.3471, 0.9884, 0.6859, 0.6714,
        1.5286, 1.1423, 0.7222, 0.8848, 1.4915, 0.5261, 0.6724, 0.7562, 1.3607,
        0.2688, 1.8160, 1.1118, 0.5817, 0.8911, 1.5470, 0.9284, 1.4022, 0.3211,
        0.7805, 0.3566, 0.7487, 1.1619, 1.1347, 1.3580, 1.0779, 0.9561, 0.7349,
        0.1853, 1.3280, 0.5563, 1.0959, 1.1058, 1.5512, 1.2634, 1.0623, 0.9049,
        1.0813, 0.6733, 0.6099, 0.7811, 2.0137, 0.5007, 0.4348, 1.4182, 0.7736,
        0.9781, 0.8759, 0.6814, 0.3576, 1.0184, 0.2800, 1.4223, 0.8915, 0.6941,
        1.1333, 0.8379, 1.8258, 0.5853, 1.0714, 0.9740, 0.9794, 1.4452, 1.2711,
        1.6786, 1.4298, 1.2954, 1.1850, 